### Builds annotation and training image json files for inference

In [ ]:
from pycocotools.coco import COCO
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import pandas as pd
import numpy as np
import yaml
%matplotlib inline
from utils import utils

In [ ]:
train_image_data = utils.read_train_image_data()
# eval_image_data = utils.read_eval_image_data()
annotation_data = utils.read_annotation_data()

In [ ]:
coco = COCO("../object_detection/train.json")
train_image_data = pd.DataFrame.from_dict(coco.imgs, orient='index').set_index('id')
annotation_data = pd.DataFrame.from_dict(coco.anns, orient='index').set_index('id')

In [ ]:
train_image_data

In [ ]:
annotation_data

## Read json containing species habitat

In [ ]:
shallow_df = pd.read_json('../shallow_counts.json')
shallow_df

In [ ]:
# img_data_keys = train_image_data.keys().tolist()
# ann_data_keys = annotation_data.keys().tolist()

In [ ]:
ann_df = annotation_data.copy()
ann_df = pd.merge(ann_df, shallow_df, how='right', on='category_id')
ann_df.index = np.arange(1, len(ann_df) + 1)
ann_df = ann_df.reset_index().rename(columns={'index': 'id'}).set_index('id')
ann_df

In [ ]:
train_image_data['image_id'] = train_image_data.index
train_image_data


In [ ]:
train_df = train_image_data.copy()
train_df = train_df[train_df.image_id.isin(ann_df.image_id)].reset_index(drop=True)
train_df.index = np.arange(1, len(train_df) + 1)
train_df = train_df.reset_index().rename(columns={'index': 'id'}).set_index('id')
train_df

about 1/3 of the training annotations are also present in the deep water

In [ ]:
annotation_data

list the images that contain only category_id that are not in the top 65

In [ ]:
df = annotation_data[annotation_data.category_id.isin(ann_df.category_id)].reset_index(drop=True)
df.index = np.arange(1, len(df) + 1)
df = df.reset_index().rename(columns={'index': 'id'}).set_index('id')
df

In [ ]:
df['original_cat_id'] = df['category_id'].copy()
df

In [ ]:
shallow_df['id'] = shallow_df.index

In [ ]:
shallow_df

In [ ]:
smap = shallow_df[['category_id', 'id']].set_index('category_id').to_dict()['id']
smap

In [ ]:
utils.remap(cat_lst, smap)

In [ ]:
def remap_ann(cat_id):
    return smap.get(cat_id)

In [ ]:
df.category_id = df.category_id.apply(remap_ann)

In [ ]:
df

Check that the number of images is correct

In [ ]:
df.image_id.unique().__len__() == train_df.__len__()

Commenting out so run all does not kill previous work

In [ ]:
df

In [ ]:
# df.reset_index().to_json('../annotation65.json')
# ann_df.reset_index().to_json('../full_anns65.json')
# train_df.reset_index().to_json('../train_image_data65.json')

## Make dataset.yaml file

In [ ]:
shallow = shallow_df.copy()
shallow['id'] = np.arange(len(shallow))
shallow


In [ ]:
shallow_map = shallow[['id', 'category']].set_index('id').to_dict()['category']
dataset = {'path': '/data/dataset',
           'train': 'images/train',
           'val': 'images/val',
           'names': shallow_map}
dataset

In [ ]:
# with open("/data/dataset65.yaml", 'w') as yamlfile:
#     data = yaml.dump(dataset, yamlfile)
#     print("Write successful")

In [ ]:
train_df.reset_index()

In [ ]:
ann_df.reset_index()

In [ ]:
df = pd.read_json('../annotation65.json')
df